In [12]:
from pyquil.quil import Program
import pyquil.api as api
from pyquil.gates import *
import numpy as np

In [13]:
#Algoritmo de Deutsch
p = Program()

# Define U_f for each of (a), (b), (c) and (d)
p.defgate("U_f_a", np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]))
p.defgate("U_f_b", np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]))
p.defgate("U_f_c", np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]]))
p.defgate("U_f_d", np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]))

# Prepare the starting state |0> x (1/sqrt[2]) *  (|0> - |1>)
p.inst(I(1))
p.inst(X(0))
p.inst(H(0))

# Apply H
p.inst(H(1))

# Apply U_f (oraculo)
p.inst(("U_f_d", 1, 0))

# Apply final H gate
p.inst(H(1))

quantum_simulator = api.QVMConnection()
wavefunc = quantum_simulator.wavefunction(p)
print (wavefunc)
print (wavefunc.get_outcome_probs())

# La salida da a '10' un 0% de probabilidad y a '11' otro 50%, es correcta ya que hemos implementado un oraculo
# f(x) de 1 y 0 en cada respectivo qubit, ademas hemos definido las puertas de la siguiente manera
# (a) f(0) = 0 ; f(1) = 0
# (b) f(0) = 1 ; f(1) = 1
# (c) f(0) = 0 ; f(1) = 1
# (d) f(0) = 1 ; f(1) = 0


(-0.7071067812+0j)|10> + (0.7071067812+0j)|11>
{'00': 0.0, '01': 0.0, '10': 0.4999999999999998, '11': 0.4999999999999998}


In [14]:
import itertools
import numpy as np
import pyquil.api as api
from pyquil.gates import *
from pyquil.quil import Program

In [15]:
def qubit_strings(n):
    qubit_strings = []
    for q in itertools.product(['0', '1'], repeat=n):
        qubit_strings.append(''.join(q))
    return qubit_strings

qubit_strings(3)

#creamos todas las combinaciones posibles

['000', '001', '010', '011', '100', '101', '110', '111']

In [16]:
# f(x), en n qubits representados por el vector x

def black_box_map(n, balanced):
    qubs = qubit_strings(n)

    # assign a constant value to all inputs if not balanced
    if not balanced:
        const_value = np.random.choice([0, 1])
        d_blackbox = {q: const_value for q in qubs}

    # assign 0 to half the inputs, and 1 to the other inputs if balanced
    if balanced:
        # randomly pick half the inputs
        half_inputs = np.random.choice(qubs, size=int(len(qubs)/2), replace=False)
        d_blackbox = {q_half: 0 for q_half in half_inputs}
        d_blackbox_other_half = {q_other_half: 1 for q_other_half in set(qubs) - set(half_inputs)}
        d_blackbox.update(d_blackbox_other_half)
    
    return d_blackbox



In [17]:
# Base Ket a partir de una cadena de n bits especificada por la entrada 'qub_string', p. Ej. '001' -> | 001>

def qubit_ket(qub_string):
    e0 = np.array([[1], [0]])
    e1 = np.array([[0], [1]])
    d_qubstring = {'0': e0, '1': e1}

    # initialize ket
    ket = d_qubstring[qub_string[0]]
    for i in range(1, len(qub_string)):
        ket = np.kron(ket, d_qubstring[qub_string[i]])
    
    return ket

In [18]:
# Crea un operador de proyeccion a partir del elemento base especificado por 'qub_string', p. Ej.
# '101' -> | 101> <101 |

def projection_op(qub_string):
    ket = qubit_ket(qub_string)
    bra = np.transpose(ket)  # all entries real, so no complex conjugation necessary
    proj = np.kron(ket, bra)
    return proj



In [19]:
# Representacion unitaria del operador de caja negra en (n + 1) -qubits

def black_box(n, balanced):

    d_bb = black_box_map(n, balanced=balanced)
    # initialize unitary matrix
    N = 2**(n+1)
    unitary_rep = np.zeros(shape=(N, N))
    # populate unitary matrix
    for k, v in d_bb.items():
        unitary_rep += np.kron(projection_op(k), np.eye(2) + v*(-np.eye(2) + np.array([[0, 1], [1, 0]])))
        
    return unitary_rep



In [23]:
#2 qubits funcion constante
p = Program()

# pick number of control qubits to be used
n = 2

# Define U_f
p.defgate("U_f", black_box(n, balanced=False))

# Prepare the starting state |0>^(\otimes n) x (1/sqrt[2])*(|0> - |1>)
for n_ in range(1, n+1):
    p.inst(I(n_))
p.inst(X(0))
p.inst(H(0))

# Apply H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Apply U_f
p.inst(("U_f",) + tuple(range(n+1)[::-1]))

# Apply final H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Final measurement
classical_regs = list(range(n))
for i, n_ in enumerate(list(range(1, n+1))[::-1]):
    p.measure(n_, classical_regs[i])

qvm = api.QVMConnection()
measure_n_qubits = qvm.run(p, classical_regs)

# flatten out list
measure_n_qubits = [item for sublist in measure_n_qubits for item in sublist]

# Determine if function is balanced or not
if set(measure_n_qubits) == set([0, 1]):
    print ("Function is balanced")
elif set(measure_n_qubits) == set([0]):
    print ("Function is constant")

Function is constant


In [24]:
#3 qubits funcion constante
p = Program()

# pick number of control qubits to be used
n = 3

# Define U_f
p.defgate("U_f", black_box(n, balanced=False))

# Prepare the starting state |0>^(\otimes n) x (1/sqrt[2])*(|0> - |1>)
for n_ in range(1, n+1):
    p.inst(I(n_))
p.inst(X(0))
p.inst(H(0))

# Apply H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Apply U_f
p.inst(("U_f",) + tuple(range(n+1)[::-1]))

# Apply final H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Final measurement
classical_regs = list(range(n))
for i, n_ in enumerate(list(range(1, n+1))[::-1]):
    p.measure(n_, classical_regs[i])

qvm = api.QVMConnection()
measure_n_qubits = qvm.run(p, classical_regs)

# flatten out list
measure_n_qubits = [item for sublist in measure_n_qubits for item in sublist]

# Determine if function is balanced or not
if set(measure_n_qubits) == set([0, 1]):
    print ("Function is balanced")
elif set(measure_n_qubits) == set([0]):
    print ("Function is constant")

Function is constant


In [25]:
#2 qubits funcion constante
p = Program()

# pick number of control qubits to be used
n = 2

# Define U_f
p.defgate("U_f", black_box(n, balanced=True))

# Prepare the starting state |0>^(\otimes n) x (1/sqrt[2])*(|0> - |1>)
for n_ in range(1, n+1):
    p.inst(I(n_))
p.inst(X(0))
p.inst(H(0))

# Apply H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Apply U_f
p.inst(("U_f",) + tuple(range(n+1)[::-1]))

# Apply final H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Final measurement
classical_regs = list(range(n))
for i, n_ in enumerate(list(range(1, n+1))[::-1]):
    p.measure(n_, classical_regs[i])

qvm = api.QVMConnection()
measure_n_qubits = qvm.run(p, classical_regs)

# flatten out list
measure_n_qubits = [item for sublist in measure_n_qubits for item in sublist]

# Determine if function is balanced or not
if set(measure_n_qubits) == set([0, 1]):
    print ("Function is balanced")
elif set(measure_n_qubits) == set([0]):
    print ("Function is constant")

Function is balanced


In [26]:
#2 qubits funcion constante
p = Program()

# pick number of control qubits to be used
n = 3

# Define U_f
p.defgate("U_f", black_box(n, balanced=True))

# Prepare the starting state |0>^(\otimes n) x (1/sqrt[2])*(|0> - |1>)
for n_ in range(1, n+1):
    p.inst(I(n_))
p.inst(X(0))
p.inst(H(0))

# Apply H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Apply U_f
p.inst(("U_f",) + tuple(range(n+1)[::-1]))

# Apply final H^(\otimes n)
for n_ in range(1, n+1):
    p.inst(H(n_))

# Final measurement
classical_regs = list(range(n))
for i, n_ in enumerate(list(range(1, n+1))[::-1]):
    p.measure(n_, classical_regs[i])

qvm = api.QVMConnection()
measure_n_qubits = qvm.run(p, classical_regs)

# flatten out list
measure_n_qubits = [item for sublist in measure_n_qubits for item in sublist]

# Determine if function is balanced or not
if set(measure_n_qubits) == set([0, 1]):
    print ("Function is balanced")
elif set(measure_n_qubits) == set([0]):
    print ("Function is constant")

Function is balanced


In [ ]:
# Como podemos observar, primero determinamos el oraculo que queremos implementar, al ser una funcion parametrizada
# es muy facil poner el numero de qubits que queremos que tenga nuestra implementacion, y tambien tenemos
# parametrizado si queremos que sea balanceada o constante, para así luego simplemente comprobarlo al final con
# la funcion set(), que comprueba si es todo 0s (constante) o no (balanceada).
# La funcion implementada f: {0, 1} ^ n -> {0, 1} puede ser constante (igual al mismo valor para todas sus 
# entradas) o balanceada (igual a un valor para exactamente la mitad de sus entradas y el otro valor para
# precisamente la otra mitad de sus entradas).